In [32]:
from openbb_terminal.sdk import openbb
import pandas as pd
import numpy as np
import datetime as dt

In [33]:
# enter stock symbol and option expiration date
stock_symbol = 'GOOG'
option_expiration_date = '2023-04-14'

In [34]:
# using top of book to get stock spot price by averaging top bid and ask
bid,ask = openbb.stocks.tob(stock_symbol)
quote_tob = bid.join(ask, lsuffix= ': Bid', rsuffix = ': Ask')
spot_price_df = pd.DataFrame(quote_tob)
spot_price = round((spot_price_df.loc[0][1] + spot_price_df.loc[0][3]) / 2,2)

In [35]:
# pulling option chain
option_chain = openbb.stocks.options.chains(symbol = stock_symbol, expiration = option_expiration_date)
option_chain_df = pd.DataFrame(option_chain)

In [36]:
# narrowing option chain to puts out of the money with a bid
put_options = option_chain_df[(option_chain_df["optionType"] == 'put')]
option_put_df = pd.DataFrame(put_options)
otm_puts = option_put_df[(option_put_df["strike"] < spot_price)]
otm_puts_df = pd.DataFrame(otm_puts)
bidded_puts = otm_puts_df[(otm_puts_df["bid"] > 0)]
bidded_puts_df = pd.DataFrame(bidded_puts)

In [37]:
# adding option price by averaging the bid and ask
bidded_puts_df.loc[:, ["optionPrice"]] = round((bidded_puts_df["bid"] + bidded_puts_df["ask"]) / 2,2)

In [38]:
# adding the spot price
bidded_puts_df.loc[:, ["spotPrice"]] = spot_price

In [52]:
# calculating the option price as a percentage of the spot price
bidded_puts_df.loc[:, ["optionPercent"]] = round(bidded_puts_df["optionPrice"] / bidded_puts_df["spotPrice"],3)

In [53]:
# looking for 1% options, but returning those greater than 0.8%
one_percent_puts = bidded_puts_df[(bidded_puts_df["optionPercent"] > .008)]
one_percent_puts

,optionType,expiration,strike,lastPrice,bid,ask,openInterest,volume,optionPrice,spotPrice,optionPercent
183,put,2023-04-14,103.0,1.10,1.08,1.09,908,596,1.08,105.08,0.010
185,put,2023-04-14,104.0,1.41,1.41,1.42,1659,857,1.42,105.08,0.014
187,put,2023-04-14,105.0,1.81,1.81,1.82,759,887,1.82,105.08,0.017
